In [ ]:
from google.colab import drive
drive.mount('content/drive')

In [ ]:
!pip install -q ultralytics roboflow

In [ ]:
#if using imported dataset from roboflow 
import os 
ROBOFLOW_API_KEY = "ENTER_YOUR_OWN_API_KEY"
os.environ["ROBOFLOW_API_KEY"] = ROBOFLOW_API_KEY

In [ ]:
#used for local data files 
import yaml
DATA_YAML = "/content/drive/MyDrive/arc_segmented/data.yaml"
names = yaml.safe_load(open(DATA_YAML))["names"]
print("names:", names)

# robust mapping (handles synonyms)
def find_id(options):
    for i,n in enumerate(names):
        if n.lower() in options: return i
    raise ValueError(f"None of {options} in names: {names}")

THREE_POINT  = find_id({"three_point_line"})
print("three_point_line =", THREE_POINT)
#for the big dataset, 3pt_line class = 0
#for the big dataset, ball class = 0, rim class = 3

In [ ]:
#use yolo segment training to train the model to detect arcs segmentations and not box identification
!yolo task=segment mode=train model=yolov8n-seg.pt data=/content/drive/MyDrive/arc_segmented/data.yaml epochs=20 imgsz=640

In [ ]:
from IPython.display import Image, display
Image(filename=f'/content/runs/segment/train/confusion_matrix.png', width=600)

In [ ]:
#training results graphs 
Image(filename=f'/content/runs/segment/train/results.png', width=600)

In [ ]:
#choose the best model that we have during training and specify the data location of our validation set and run over the validation set 
!yolo task=segment mode=val model=/content/runs/segment/train/weights/best.pt data=/content/drive/MyDrive/arc_segmented/data.yaml

In [ ]:
#now perform prediction on the trained model on the test set
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.5 source=/content/drive/MyDrive/arc_segmented/test/images

In [ ]:
import glob
#view a few of the segmentations 
#ouput shows both the box and the segmentation of the arc use the arc mask 
i =0
for image_path in glob.glob(f'/content/runs/segment/predict/*.jpg'):
    if i == 10:
        break
    display(Image(filename=image_path, height=600))
    print("/n")
    i+=1